In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Load train and test data
train_ids = ['001', '005', '006', '009', '010', '011', '012', '013', '015', '021', '022', '023',
             '024', '026', '028', '031', '032', '033', '036', '038', '041', '042', '044',
             '045', '047', '053', '054', '055', '060', '063', '064', '066', '070', '072',
             '073', '075', '076', '077', '078']
test_ids = ['002', '003', '007', '017', '018', '019', '020', '025', '029', '030', '039', '043',
            '048', '052', '059', '065', '068', '074']

In [9]:
video_id_file = "video_id.csv"
video_id_df = pd.read_csv(video_id_file)
# 두 번째 열을 'emotion' 열로 대체
video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
emotions = video_id_df['emotion'].tolist()
print(emotions)

['neutral', 'disgust', 'fear', 'sad', 'happy', 'anger', 'neutral', 'anger', 'disgust', 'happy', 'sad', 'fear', 'fear', 'happy', 'sad', 'neutral', 'disgust', 'anger', 'neutral', 'anger', 'sad', 'disgust', 'fear', 'happy', 'fear', 'happy', 'sad', 'neutral', 'anger', 'disgust', 'fear', 'sad', 'anger', 'neutral', 'happy', 'disgust', 'happy', 'disgust', 'fear', 'anger', 'sad', 'neutral', 'anger', 'neutral', 'fear', 'sad', 'disgust', 'happy', 'happy', 'neutral', 'anger', 'fear', 'sad', 'disgust', 'anger', 'fear', 'neutral', 'disgust', 'sad', 'happy', 'disgust', 'sad', 'neutral', 'happy', 'fear', 'anger', 'sad', 'fear', 'neutral', 'happy', 'disgust', 'anger', 'fear', 'sad', 'disgust', 'happy', 'neutral', 'anger', 'neutral', 'disgust', 'anger', 'happy', 'sad', 'fear', 'fear', 'neutral', 'disgust', 'anger', 'sad', 'happy', 'sad', 'happy', 'disgust', 'anger', 'fear', 'neutral', 'happy', 'neutral', 'disgust', 'sad', 'fear', 'anger', 'happy', 'sad', 'neutral', 'fear', 'disgust', 'anger', 'happy', 

In [5]:
def get_max_sequence_length(ids):
    max_len = 0
    for id in ids:
        video_id_file = "video_id.csv"
        video_id_df = pd.read_csv(video_id_file)
        video_id_df['video_id'] = video_id_df['video_id'].apply(lambda x: str(x).zfill(3))
        video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
        emotions = video_id_df[video_id_df.video_id == id]['emotion'].tolist()
        for emotion in emotions:
            file_path = f"{id}/{id}_{emotion}.csv"
            data = pd.read_csv(file_path, header=None)
            sequence_length = len(data)
            if sequence_length > max_len:
                max_len = sequence_length
    return max_len 

# 최대 시퀀스 길이 확인
max_sequence_length = get_max_sequence_length(train_ids)
print(max_sequence_length)

13690


In [6]:
def load_and_preprocess_data(ids, max_len=max_sequence_length):
    features = []
    scaler = StandardScaler()
    for id in ids:
        # video_id.csv 파일 가져오기, id 형식 변환
        video_id_file = "video_id.csv"
        video_id_df = pd.read_csv(video_id_file)
        video_id_df['video_id'] = video_id_df['video_id'].apply(lambda x: str(x).zfill(3))
        # 두 번째 열을 'emotion' 열로 대체
        video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
        # 해당 id의 감정 순서 리스트 생성
        emotions = video_id_df[video_id_df.video_id == id]['emotion'].tolist()
        for emotion in emotions:
            # 감정에 해당하는 파일 경로 탐색
            file_path = f"{id}/{id}_{emotion}.csv"
            data = pd.read_csv(file_path, header=None)
            # 데이터 2D 배열로 변환
            data = data.values.reshape(-1, 1)
            # 데이터 표준화
            data = scaler.fit_transform(data)
            # 데이터를 max_len으로 잘라냄
            data = data[:max_len]
            # 만약 데이터의 길이가 max_len보다 작다면, 나머지 부분을 패딩
            if len(data) < max_len:
                data = np.pad(data, ((0, max_len - len(data)), (0, 0)), 'constant')
            # 데이터 추가
            features.append(data)

    features_padded = np.array(features)
    return features_padded

train_features = load_and_preprocess_data(train_ids)

In [7]:
print(train_features.shape)

(234, 13690, 1)


In [8]:
print(train_features)

[[[ 2.93475448]
  [ 2.99404729]
  [ 2.96438931]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[-0.50565603]
  [-0.51482795]
  [-0.51482795]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 2.53794074]
  [ 2.57023986]
  [ 2.57023986]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 ...

 [[-1.16702695]
  [-1.16702695]
  [ 0.06997831]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 2.6772027 ]
  [ 0.17921862]
  [ 0.17921862]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[-0.56540335]
  [-1.82864931]
  [-0.56540335]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]]


In [9]:
def create_autoencoder(input_shape, encoding_dim):
    # Input layer
    input_layer = Input(shape=input_shape)

    # Flatten layer: the Dense layer works on 2D inputs.
    flat_layer = Flatten()(input_layer)

    # Encoding layer
    encoded = Dense(encoding_dim, activation='relu')(flat_layer)

    # Decoding layer
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(encoded)

    # Reshape layer: to match the original input shape.
    reshaped = Reshape(input_shape)(decoded)

    # Autoencoder model
    autoencoder = Model(input_layer, reshaped)
    
    # Encoder model
    encoder = Model(input_layer, encoded)

    return autoencoder, encoder

# Encoding dimension
encoding_dim = 64  # or any other value

# Autoencoder model
autoencoder, encoder = create_autoencoder((max_sequence_length, 1), encoding_dim)

In [10]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(train_features, train_features, epochs=50, batch_size=128, shuffle=True)

# Extracting features
train_features_encoded = encoder.predict(train_features)

Epoch 1/50
2/2 [==============================] - 1s 73ms/step - loss: 0.3968
Epoch 2/50
2/2 [==============================] - 0s 62ms/step - loss: 0.3854
Epoch 3/50
2/2 [==============================] - 0s 64ms/step - loss: 0.3526
Epoch 4/50
2/2 [==============================] - 0s 63ms/step - loss: 0.3003
Epoch 5/50
2/2 [==============================] - 0s 66ms/step - loss: 0.2465
Epoch 6/50
2/2 [==============================] - 0s 75ms/step - loss: 0.2059
Epoch 7/50
2/2 [==============================] - 0s 76ms/step - loss: 0.1814
Epoch 8/50
2/2 [==============================] - 0s 81ms/step - loss: 0.1681
Epoch 9/50
2/2 [==============================] - 0s 80ms/step - loss: 0.1606
Epoch 10/50
2/2 [==============================] - 0s 97ms/step - loss: 0.1555
Epoch 11/50
2/2 [==============================] - 0s 74ms/step - loss: 0.1523
Epoch 12/50
2/2 [==============================] - 0s 79ms/step - loss: 0.1498
Epoch 13/50
2/2 [==============================] - 0s 79ms/st

In [11]:
print(train_features_encoded.shape)

(234, 64)


In [12]:
train_features_encoded

array([[12.550317,  0.      , 21.81655 , ..., 21.984102, 21.86208 ,
        13.78929 ],
       [ 0.      , 14.353313,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [14.540529,  0.      , 24.586775, ..., 24.902727, 23.910677,
        16.990757],
       ...,
       [ 0.      , 33.022785,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [41.18801 ,  0.      ,  0.      , ...,  0.      , 22.506584,
         0.      ],
       [ 0.      , 19.07907 , 37.35222 , ..., 30.521257, 30.937357,
         0.      ]], dtype=float32)

In [13]:
# Convert the encoded features to a DataFrame
df_encoded = pd.DataFrame(train_features_encoded)

# Save the DataFrame to a CSV file
output_file = "encoded_features_EDA.csv"
df_encoded.to_csv(output_file, index=False)

In [10]:
video_id_file = "video_id_test.csv"
video_id_df = pd.read_csv(video_id_file)
# 두 번째 열을 'emotion' 열로 대체
video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
emotions = video_id_df['emotion'].tolist()
print(emotions)

['anger', 'disgust', 'neutral', 'happy', 'sad', 'fear', 'happy', 'neutral', 'anger', 'sad', 'disgust', 'fear', 'disgust', 'sad', 'happy', 'fear', 'neutral', 'anger', 'disgust', 'neutral', 'fear', 'anger', 'sad', 'happy', 'anger', 'fear', 'sad', 'disgust', 'neutral', 'happy', 'anger', 'disgust', 'neutral', 'happy', 'sad', 'fear', 'happy', 'fear', 'anger', 'neutral', 'disgust', 'sad', 'fear', 'happy', 'sad', 'neutral', 'disgust', 'anger', 'anger', 'disgust', 'sad', 'happy', 'fear', 'neutral', 'disgust', 'anger', 'neutral', 'fear', 'sad', 'happy', 'neutral', 'disgust', 'happy', 'sad', 'anger', 'fear', 'anger', 'sad', 'fear', 'happy', 'neutral', 'disgust', 'disgust', 'neutral', 'fear', 'happy', 'sad', 'anger', 'sad', 'neutral', 'anger', 'disgust', 'fear', 'happy', 'disgust', 'fear', 'sad', 'neutral', 'anger', 'happy', 'neutral', 'happy', 'fear', 'sad', 'disgust', 'anger', 'neutral', 'happy', 'anger', 'fear', 'sad', 'disgust', 'neutral', 'anger', 'happy', 'disgust', 'sad', 'fear']


In [12]:
def get_max_sequence_length(ids):
    max_len = 0
    for id in ids:
        video_id_file = "video_id_test.csv"
        video_id_df = pd.read_csv(video_id_file)
        video_id_df['video_id'] = video_id_df['video_id'].apply(lambda x: str(x).zfill(3))
        video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
        emotions = video_id_df[video_id_df.video_id == id]['emotion'].tolist()
        for emotion in emotions:
            file_path = f"{id}/{id}_{emotion}.csv"
            data = pd.read_csv(file_path, header=None)
            sequence_length = len(data)
            if sequence_length > max_len:
                max_len = sequence_length
    return max_len 

# 최대 시퀀스 길이 확인
max_sequence_length_test = get_max_sequence_length(test_ids)
print(max_sequence_length_test)

27226


In [13]:
def load_and_preprocess_data(ids, max_len=max_sequence_length_test):
    features = []
    scaler = StandardScaler()
    for id in ids:
        # video_id_test.csv 파일 가져오기, id 형식 변환
        video_id_file = "video_id_test.csv"
        video_id_df = pd.read_csv(video_id_file)
        video_id_df['video_id'] = video_id_df['video_id'].apply(lambda x: str(x).zfill(3))
        # 두 번째 열을 'emotion' 열로 대체
        video_id_df = video_id_df.rename(columns={video_id_df.columns[1]: 'emotion'})
        # 해당 id의 감정 순서 리스트 생성
        emotions = video_id_df[video_id_df.video_id == id]['emotion'].tolist()
        for emotion in emotions:
            # 감정에 해당하는 파일 경로 탐색
            file_path = f"{id}/{id}_{emotion}.csv"
            data = pd.read_csv(file_path, header=None)
            # 데이터 2D 배열로 변환
            data = data.values.reshape(-1, 1)
            # 데이터 표준화
            data = scaler.fit_transform(data)
            # 데이터를 max_len으로 잘라냄
            data = data[:max_len]
            # 만약 데이터의 길이가 max_len보다 작다면, 나머지 부분을 패딩
            if len(data) < max_len:
                data = np.pad(data, ((0, max_len - len(data)), (0, 0)), 'constant')
            # 데이터 추가
            features.append(data)

    features_padded = np.array(features)
    return features_padded

test_features = load_and_preprocess_data(test_ids)

In [14]:
print(test_features.shape)

(108, 27226, 1)


In [15]:
print(test_features)

[[[-1.13545188]
  [-1.13545188]
  [-1.13545188]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.        ]
  [ 0.        ]
  [ 0.        ]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 ...

 [[-1.16463266]
  [-1.14347553]
  [-1.14347553]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 1.55791807]
  [ 1.56399168]
  [ 1.56196609]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]

 [[ 0.25330206]
  [ 0.25198198]
  [ 0.24802071]
  ...
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]]]


In [16]:
def create_autoencoder(input_shape, encoding_dim):
    # Input layer
    input_layer = Input(shape=input_shape)

    # Flatten layer: the Dense layer works on 2D inputs.
    flat_layer = Flatten()(input_layer)

    # Encoding layer
    encoded = Dense(encoding_dim, activation='relu')(flat_layer)

    # Decoding layer
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(encoded)

    # Reshape layer: to match the original input shape.
    reshaped = Reshape(input_shape)(decoded)

    # Autoencoder model
    autoencoder = Model(input_layer, reshaped)
    
    # Encoder model
    encoder = Model(input_layer, encoded)

    return autoencoder, encoder

# Encoding dimension
encoding_dim = 64  # or any other value

# Autoencoder model
autoencoder, encoder = create_autoencoder((max_sequence_length_test, 1), encoding_dim)

In [17]:
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(test_features, test_features, epochs=50, batch_size=128, shuffle=True)

# Extracting features
test_features_encoded = encoder.predict(test_features)

Epoch 1/50
1/1 [==============================] - 2s 2s/step - loss: 0.3549
Epoch 2/50
1/1 [==============================] - 0s 117ms/step - loss: 0.3538
Epoch 3/50
1/1 [==============================] - 0s 136ms/step - loss: 0.3500
Epoch 4/50
1/1 [==============================] - 0s 130ms/step - loss: 0.3406
Epoch 5/50
1/1 [==============================] - 0s 133ms/step - loss: 0.3240
Epoch 6/50
1/1 [==============================] - 0s 124ms/step - loss: 0.3010
Epoch 7/50
1/1 [==============================] - 0s 124ms/step - loss: 0.2738
Epoch 8/50
1/1 [==============================] - 0s 117ms/step - loss: 0.2452
Epoch 9/50
1/1 [==============================] - 0s 122ms/step - loss: 0.2179
Epoch 10/50
1/1 [==============================] - 0s 144ms/step - loss: 0.1937
Epoch 11/50
1/1 [==============================] - 0s 133ms/step - loss: 0.1740
Epoch 12/50
1/1 [==============================] - 0s 149ms/step - loss: 0.1587
Epoch 13/50
1/1 [==============================] - 0

In [18]:
print(test_features_encoded.shape)

(108, 64)


In [19]:
test_features_encoded

array([[0.0000000e+00, 1.2783803e+00, 3.2280754e+01, ..., 3.9714165e+01,
        5.2484436e+01, 3.9255604e+01],
       [4.6772931e-02, 4.7429938e-02, 4.7759414e-02, ..., 4.9848713e-02,
        4.9521651e-02, 4.9831320e-02],
       [4.6772931e-02, 4.7429938e-02, 4.7759414e-02, ..., 4.9848713e-02,
        4.9521651e-02, 4.9831320e-02],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.7787821e+01,
        0.0000000e+00, 1.7016611e+01],
       [1.1985534e+01, 2.4901291e+01, 2.6198778e+01, ..., 0.0000000e+00,
        2.3725740e+01, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 2.3570986e+01,
        0.0000000e+00, 2.3624538e+01]], dtype=float32)

In [20]:
# Convert the encoded features to a DataFrame
df_encoded = pd.DataFrame(test_features_encoded)

# Save the DataFrame to a CSV file
output_file = "encoded_features_test_EDA.csv"
df_encoded.to_csv(output_file, index=False)